In [1]:
import cffi
ffi = cffi.FFI()

In [2]:
ffi.cdef("""
    typedef struct ANDORCAPS
    {
      ULONG ulSize;
        ULONG ulAcqModes;
        ULONG ulReadModes;
        ULONG ulTriggerModes;
        ULONG ulCameraType;
        ULONG ulPixelMode;
        ULONG ulSetFunctions;
        ULONG ulGetFunctions;
        ULONG ulFeatures;
        ULONG ulPCICard;
        ULONG ulEMGainCapability;
        ULONG ulFTReadModes;
    } AndorCapabilities;
""")

In [6]:
lib = ffi.dlopen?

In [ ]:
lib = ffi.dlopen

In [7]:
lib.Initialize

AttributeError: Initialize

# ----------------------------------------------

In [8]:
del cffi, ffi, lib

In [1]:
import ctypes
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class ANDORCAPS(ctypes.Structure):
    _fields_ = [("ulSize", ctypes.c_ulong),
                ("ulAcqModes", ctypes.c_ulong),
                ("ulReadModes", ctypes.c_ulong),
                ("ulTriggerModes", ctypes.c_ulong),
                ("ulCameraType", ctypes.c_ulong),
                ("ulPixelMode", ctypes.c_ulong),
                ("ulSetFunctions", ctypes.c_ulong),
                ("ulGetFunctions", ctypes.c_ulong),
                ("ulFeatures", ctypes.c_ulong),
                ("ulPCICard", ctypes.c_ulong),
                ("ulEMGainCapability", ctypes.c_ulong),
                ("UlFTReadModes", ctypes.c_ulong)]

In [3]:
lib = ctypes.cdll.LoadLibrary("atmcd64d.dll")

In [4]:
n_cams = ctypes.c_int()
print lib.GetAvailableCameras(ctypes.byref(n_cams))
print n_cams

20002
c_long(2)


In [5]:
newton_handle = ctypes.c_int()
idus_handle = ctypes.c_int()
print lib.GetCameraHandle(0, ctypes.byref(newton_handle))
print newton_handle
print lib.GetCameraHandle(1, ctypes.byref(idus_handle))
print idus_handle

20002
c_long(100)
20002
c_long(201)


In [6]:
print lib.SetCurrentCamera(newton_handle)
print lib.Initialize()
print lib.SetCurrentCamera(idus_handle)
print lib.Initialize()

20002
20992
20002
20002


In [11]:
#c_dir = ctypes.c_char_p(".")
#lib.Initialize(c_dir)

In [12]:
newton_ser_num = ctypes.c_int()
idus_ser_num = ctypes.c_int()
print lib.SetCurrentCamera(newton_handle)
print lib.GetCameraSerialNumber(ctypes.byref(newton_ser_num))
print newton_ser_num
print lib.SetCurrentCamera(idus_handle)
print lib.GetCameraSerialNumber(ctypes.byref(idus_ser_num))
print idus_ser_num

20002
20002
c_long(17910)
20002
20002
c_long(17911)


In [13]:
print lib.SetCurrentCamera(newton_handle)

20002


In [14]:
xpixels, ypixels = ctypes.c_int(), ctypes.c_int()
ret = lib.GetDetector(ctypes.byref(xpixels), ctypes.byref(ypixels))
print "ret: %d" % ret
print "screen: %d by %d" % (xpixels.value, ypixels.value)

ret: 20002
screen: 1024 by 256


In [15]:
args = [ctypes.byref(ctypes.c_int()) for i in xrange(6)]
print lib.GetHardwareVersion(*args)
for i in args:
    print i._obj # dereference pointer

20002
c_long(0)
c_long(0)
c_long(0)
c_long(131)
c_long(20)
c_long(24)


In [16]:
n_VS_speeds = ctypes.c_int()
print lib.GetNumberVSSpeeds(ctypes.byref(n_VS_speeds))
print n_VS_speeds

20002
c_long(11)


In [18]:
c_i = ctypes.c_int()
speed = ctypes.c_float()
VS_speeds = []
for i in xrange(n_VS_speeds.value):
    c_i.value = i
    ret = lib.GetVSSpeed(c_i, ctypes.byref(speed))
    print "ret: %d; speed: %f" % (ret, speed.value)
    VS_speeds.append(speed.value)

ret: 20002; speed: 1.700000
ret: 20002; speed: 3.300000
ret: 20002; speed: 6.500000
ret: 20002; speed: 12.900000
ret: 20002; speed: 25.700001
ret: 20002; speed: 51.299999
ret: 20002; speed: 76.900002
ret: 20002; speed: 102.500000
ret: 20002; speed: 128.100006
ret: 20002; speed: 153.699997
ret: 20002; speed: 179.300003


In [19]:
args = [ctypes.byref(ctypes.c_int()) for i in xrange(6)]
print lib.GetSoftwareVersion(*args)
for i in args:
    print i._obj

20002
c_long(0)
c_long(0)
c_long(0)
c_long(0)
c_long(99)
c_long(2)


In [22]:
n_HS_speeds = ctypes.c_int()
print lib.GetNumberHSSpeeds(ctypes.byref(n_HS_speeds))
print n_HS_speeds

20068
c_long(0)


In [17]:
c_i = ctypes.c_int()
speed = ctypes.c_float()
HS_speeds = []
for i in xrange(n_HS_speeds.value):
    c_i.value = i
    ret = lib.GetHSSpeed(c_i, ctypes.byref(speed))
    print "ret: %d; speed: %f" % (ret.value, speed.value)
    HS_speeds.append(speed.value)

below begins step 2 of the guide

In [23]:
mintemp, maxtemp = ctypes.c_int(), ctypes.c_int()
print lib.GetTemperatureRange(ctypes.byref(mintemp), ctypes.byref(maxtemp))
print mintemp, maxtemp

20002
c_long(-120) c_long(-10)


In [24]:
lib.SetTemperature(ctypes.c_int(-70))

20002

In [25]:
lib.CoolerON()

20002

below is step 3 of the guide

In [26]:
# monitor until temperature stabilized
temp = ctypes.c_int()
stabilized = False
while not stabilized:
    ret = lib.GetTemperature(ctypes.byref(temp))
    print temp.value
    print ret
    if ret == 20036: stabilized = True
    time.sleep(5)
print "stabilized!"

26
20037
24
20037
22
20037
19
20037
16
20037
12
20037
9
20037
6
20037
3
20037
0
20037
-3
20037
-6
20037
-9
20037
-12
20037
-15
20037
-18
20037
-20
20037
-24
20037
-26
20037
-28
20037
-30
20037
-33
20037
-35
20037
-37
20037
-39
20037
-41
20037
-43
20037
-45
20037
-46
20037
-48
20037
-50
20037
-51
20037
-53
20037
-55
20037
-56
20037
-57
20037
-59
20037
-60
20037
-61
20037
-62
20037
-64
20037
-64
20037
-66
20037
-66
20037
-67
20037
-68
20035
-69
20035
-70
20035
-71
20035
-72
20035
-72
20035
-72
20037
-74
20037
-74
20037
-74
20037
-76
20037
-76
20037
-76
20037
-76
20037
-76
20037
-76
20037
-76
20037
-76
20037
-76
20037
-76
20037
-75
20037
-75
20037
-74
20037
-74
20037
-74
20037
-74
20037
-74
20037
-74
20037
-73
20037
-73
20037
-73
20037
-72
20037
-72
20037
-72
20037
-72
20037
-72
20037
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-72
20035
-71
20035
-71
20035
-71
20035
-71
20035
-71
20035
-71
20035
-71
20035
-71
20035
-71
200

step 4 is checking that it's stablizied.
below is step 5 of the guide

In [27]:
lib.SetAcquisitionMode(ctypes.c_int(1))

20002

In [28]:
# lib.SetReadoutMode doesn't exist; presumably the below?
lib.SetReadMode(ctypes.c_int(0))

20002

In [29]:
typ = ctypes.c_int(0) # need to look this up! TTL low or high to open shutter?
mode = ctypes.c_int(0)
closingtime = ctypes.c_int(50) # long time, for now
openingtime = ctypes.c_int(50)
lib.SetShutter(typ, mode, closingtime, openingtime)

20002

In [59]:
lib.SetExposureTime(ctypes.c_float(.01))

20002

In [31]:
lib.SetTriggerMode(ctypes.c_int(0))
# try also software trigger

20002

In [ ]:
lib.SetAccumulationCycleTime # ignore this until accumulation mode is needed

In [ ]:
lib.SetNumberAccumulations # ignore this until accumulation mode is needed

In [ ]:
lib.SetNumberKinetics # ignore this until kinetic mode is needed

In [ ]:
lib.SetKineticCycletime # ignore this until kinetic mode is needed

In [60]:
args = [ctypes.byref(ctypes.c_float()) for i in xrange(3)]
print lib.GetAcquisitionTimings(*args)
for i in args:
    print i._obj

20002
c_float(0.009999999776482582)
c_float(0.009999999776482582)
c_float(0.009999999776482582)


In [ ]:
lib.SetHSSpeed # look this up after getting HS speeds

In [ ]:
lib.SetVSSpeed # look this up after getting VS speeds

below is step 6 of the guide

In [61]:
# the guide omitted something that the GetStatus documentation said:
# should call this before an acquisition to check that it's idle (20073)
status = ctypes.c_int()
print lib.GetStatus(ctypes.byref(status))
print status

20002
c_long(20073)


In [62]:
lib.StartAcquisition()

20002

below are steps 7 and 8 of the guide

In [63]:
# wait until Data Acquisition is complete
done = False
while not done:
    print lib.GetStatus(ctypes.byref(status))
    print status
    if status.value == 20073: done = True
    time.sleep(2)
print "done acquiring!"

20002
c_long(20073)
done acquiring!


below are steps 9 and 10 of the guide

In [64]:
#def product(l):
#    return reduce(lambda x, y: x*y, l)

data = np.zeros((xpixels.value, ypixels.value), dtype=np.int32)
#size = ctypes.c_ulong(product(data.shape))
size= ctypes.c_ulong(xpixels.value)
print lib.GetAcquiredData(data.ctypes, size)
print data.shape
print data
# get new acquisiton? if yes, go back to step 5 (change parameters) or 6 (don't)

20002
(1024L, 256L)
[[317 316 322 ..., 311 306 305]
 [300 307 311 ..., 304 303 298]
 [305 302 308 ..., 305 300 299]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]


In [65]:
plt.plot(xrange(size.value), data)
plt.show()

In [66]:
plt.imshow(data.T, interpolation=None)
plt.show()

In [53]:
data *= 0

there's actually no step 11.
below is step 12 of the guide

In [67]:
lib.CoolerOFF()

20002

below are steps 12 and 13 of the guide

In [69]:
# wait until temperature has risen sufficiently
print lib.GetTemperature(ctypes.byref(temp))
print temp

20034
c_long(-69)


below are steps 15 and 16 of the guide

In [70]:
lib.ShutDown(ctypes.c_int(0))
# close application

20002

In [98]:
# checking 1 sec vs 0.01 sec exposure times

exposure_times = [1, 0.01]
data = []

def get_exposure(t):
    print "SetExposureTime: %d" % lib.SetExposureTime(ctypes.c_float(t))
    args = [ctypes.byref(ctypes.c_float()) for i in xrange(3)]
    print "\tGetAcqusitionTimings: %d, %d" % (
        lib.GetAcquisitionTimings(*args), args[0]._obj.value)
    print "\tStartAcquisition: %d" % lib.StartAcquisition()
    done = False
    while not done:
        print lib.GetStatus(ctypes.byref(status))
        print status
        if status.value == 20073: done = True
        time.sleep(2)
    print "done acquiring!"
    data = np.zeros((xpixels.value, ypixels.value), dtype=np.int32)
    size = ctypes.c_ulong(product(data.shape))
    print "GetAcquiredData: %d" % lib.GetAcquiredData(data.ctypes, size)
    return data

for t in exposure_times:
    data.append(get_exposure(t))

SetExposureTime: 20002
	GetAcqusitionTimings: 20002, 1
	StartAcquisition: 20002
20002
c_long(20072)
20002
c_long(20073)
done acquiring!
GetAcquiredData: 20002
SetExposureTime: 20002
	GetAcqusitionTimings: 20002, 0
	StartAcquisition: 20002
20002
c_long(20072)
20002
c_long(20073)
done acquiring!
GetAcquiredData: 20002


In [100]:
plt.plot(xrange(512), data[0], xrange(512), data[1])

In [101]:
plt.show()